In [9]:
#importing required libraries
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

In [10]:
#installing BeautifulSoup library to help scrape WikiPage (if not installed before)
!pip install beautifulsoup4
#installing lxml parser for BeautifulSoup to use (if not installed before)
!pip install lxml
#installing requests to work with URLs (if not installed before)
!pip install requests

In [11]:
#importing BeautifulSoup and requests
from bs4 import BeautifulSoup
import requests

In [12]:
#load WikiPage with list of PostalCodes for Toronto
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text
soup = BeautifulSoup(source, 'lxml')

In [13]:
#cut a table from page to work with
table = soup.find('table', class_='wikitable sortable')

In [14]:
#create columns out of headers in table
column_names = []
for header in table.find_all('th'):
    column_names.append(header.text.strip())
#create new dataframe with extracted columns
df = pd.DataFrame(columns=column_names)

In [15]:
#extract rows from table based on <tr> tag
rows = table.find_all('tr')
for row in rows[1:]: #skipping zero element as it's header
    cols = row.find_all('td')
    if cols[1].text.strip() == 'Not assigned': #skipping lines with not assigned borough
        continue
    elif cols[2].text.strip() == 'St. James Town': ##skipping duplicate St James
        continue
    elif cols[2].text.strip() == 'Not assigned': #if neighbourhood not assigned make it equal to borough
        df = df.append([{'Postcode':cols[0].text.strip(), 'Borough':cols[1].text.strip(), 'Neighbourhood':cols[1].text.strip()}], ignore_index=True)
    else:
        df = df.append([{'Postcode':cols[0].text.strip(), 'Borough':cols[1].text.strip(), 'Neighbourhood':cols[2].text.strip()}], ignore_index=True)

In [16]:
df1 = pd.read_csv('https://cocl.us/Geospatial_data')

In [17]:
df1.rename(columns={'Postal Code':'Postcode'}, inplace=True)

In [18]:
df2 = pd.merge(df, df1, on='Postcode')
df2.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Heights,43.718518,-79.464763
4,M6A,North York,Lawrence Manor,43.718518,-79.464763
5,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
6,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
7,M1B,Scarborough,Rouge,43.806686,-79.194353
8,M1B,Scarborough,Malvern,43.806686,-79.194353
9,M3B,North York,Don Mills North,43.745906,-79.352188
